<a href="https://colab.research.google.com/github/Didier06/IA_FABLAB/blob/main/Introduction_IA/Yolov8_Minst_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
#   MNIST multi-chiffres avec YOLOv8m (Ultralytics)
# ======================================================

!pip install ultralytics torchvision tqdm pyyaml --quiet

from torchvision import datasets
from PIL import Image
import random, os, yaml
from tqdm import tqdm

In [ ]:
# ======================================================
# 1️⃣ Monter Google Drive pour sauvegarder le modèle
# ======================================================

from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/MNIST_YOLOv8m"
os.makedirs(save_dir, exist_ok=True)
print(f"📁 Modèle sera sauvegardé dans : {save_dir}")

In [ ]:
# ======================================================
# 6️⃣ Test de prédiction (sur ton modèle sauvegardé sur Drive)
# ======================================================
from ultralytics import YOLO
chemin = "/content/drive/MyDrive/IA/yolo_mnist_training/"
# Chemin vers le modèle entraîné (best.pt sur ton Drive)
model_path = chemin +"mnist_yolo_run/weights/best.pt"

# Charger le modèle final
model = YOLO(model_path)

# Image de test
test_image = chemin + "images/val/10.jpg"

# Lancer la prédiction et sauvegarder le résultat
results = model.predict(source=test_image, save=True, imgsz=320)

# Afficher la prédiction directement dans Colab
results[0].show()


In [ ]:
from ultralytics import YOLO
chemin = "/content/drive/MyDrive/IA/yolo_mnist_training/"
# Charger ton modèle
model = YOLO(chemin + "mnist_yolo_run/weights/best.pt")

# Faire la prédiction (sans sauvegarde automatique)
results = model.predict(source= chemin + "images/val/52.jpg", save=False, imgsz=320)

# Afficher sans les probabilités
results[0].plot(conf=False, labels=False, save=True)


In [ ]:
# ======================================================
# Test YOLOv8 — affichage sans probas + comptage visuel des classes
# ======================================================
from ultralytics import YOLO
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display
import os
chemin = "/content/drive/MyDrive/IA/yolo_mnist_training/"
# 1️⃣ Charger le modèle
model = YOLO(chemin +"mnist_yolo_run/weights/best.pt")

# 2️⃣ Lancer la prédiction sans sauvegarde automatique
nom_fichier = "100.jpg"
results = model.predict(source= chemin + "images/val/" + nom_fichier, save=False, imgsz=320)

# 3️⃣ Générer et sauvegarder l’image propre (sans probas ni labels)
output_dir = chemin +"mnist_yolo_run/predictions"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "clean_"+nom_fichier)

# La fonction plot() retourne l’image → on la sauvegarde proprement
img_clean = results[0].plot(conf=False, labels=False)
import cv2
cv2.imwrite(output_path, cv2.cvtColor(img_clean, cv2.COLOR_RGB2BGR))

# 4️⃣ Afficher l’image dans Colab
display(Image(filename=output_path))
print(f"🖼️ Image sauvegardée : {output_path}")

# ======================================================
# 5️⃣ Comptage des classes détectées
# ======================================================
classes = results[0].boxes.cls.cpu().numpy().astype(int)
counts = Counter(classes)

if counts:
    # Affichage textuel
    summary = ", ".join([f"{cls}:{count}" for cls, count in counts.items()])
    print(f"📊 Détections : {summary}")

    # Tableau de comptage
    df = pd.DataFrame(list(counts.items()), columns=["Classe", "Occurrences"])
    #display(df)

    # 6️⃣ Histogramme (barres)
    plt.figure(figsize=(6,4))
    plt.bar(df["Classe"].astype(str), df["Occurrences"], color="skyblue")
    plt.xlabel("Classe détectée")
    plt.ylabel("Occurrences")
    plt.title("Histogramme des classes détectées")
    plt.show()

else:
    print("❌ Aucune détection trouvée sur cette image.")
